In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import mplhep as hep
import matplotlib.ticker as mticker
import numpy as np

import uproot
import awkward as ak
from coffea import nanoevents

from coffea.nanoevents.methods.base import NanoEventsArray
from coffea.analysis_tools import Weights, PackedSelection
from coffea.nanoevents.methods import nanoaod
from coffea.nanoevents.methods import vector
from coffea.lookup_tools.dense_lookup import dense_lookup

from HHbbVV.processors.utils import pad_val

plt.style.use(hep.style.CMS)
hep.style.use("CMS")
formatter = mticker.ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-3, 3))
plt.rcParams.update({"font.size": 24})

import warnings

warnings.filterwarnings("ignore")

In [ ]:
from datetime import datetime
from pathlib import Path

MAIN_DIR = Path("../../../")
samples_dir = MAIN_DIR / "../data/skimmer/24Mar5AllYears"
# samples_dir = "/ceph/cms/store/user/annava/projects/HHbbVV/24Mar5AllYears"
year = "2016"

# date = "24Mar6"
date = datetime.now().strftime("%Y%m%d")
plot_dir = MAIN_DIR / f"plots/Kinematics/{date}/"
plot_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
Z_PDGID = 23
W_PDGID = 24
HIGGS_PDGID = 25
b_PDGID = 5
GEN_FLAGS = ["fromHardProcess", "isLastCopy"]

Look at single SM VBF HH signal NanoAOD file

In [ ]:
events = nanoevents.NanoEventsFactory.from_root(
    # "root://cmseos.fnal.gov///store/user/lpcpfnano/cmantill/v2_3/2018/HH/VBF_HHTobbVV_CV_1_C2V__C3_1_TuneCP5_13TeV-madgraph-pythia8/VBF_HHTobbVV_CV_1_C2V_1_C3_1/220808_150149/0000/nano_mc2018_1-1.root",
    "root://cmseos.fnal.gov///store/user/lpcpfnano/cmantill/v2_3/2018/HH/VBF_HHTobbVV_CV_1_C2V_0_C3_1_TuneCP5_13TeV-madgraph-pythia8/VBF_HHTobbVV_CV_1_C2V_0_C3_1/220808_150000/0000/nano_mc2018_1-1.root",
    schemaclass=nanoevents.NanoAODSchema,
).events()

Get generator-level Higgs and Vs

In [ ]:
higgs = events.GenPart[
    (abs(events.GenPart.pdgId) == HIGGS_PDGID) * events.GenPart.hasFlags(GEN_FLAGS)
]

higgs_children = higgs.children

# finding bb and VV children
is_bb = abs(higgs_children.pdgId) == b_PDGID
is_VV = (abs(higgs_children.pdgId) == W_PDGID) + (abs(higgs_children.pdgId) == Z_PDGID)

Hbb = higgs[ak.any(is_bb, axis=2)]
HVV = higgs[ak.any(is_VV, axis=2)]

# make sure we're only getting one Higgs
Hbb = ak.pad_none(Hbb, 1, axis=1)[:, 0]
HVV = ak.pad_none(HVV, 1, axis=1)[:, 0]

vs = events.GenPart[((abs(events.GenPart.pdgId) == 24)) * events.GenPart.hasFlags(GEN_FLAGS)]

# vbf output quarks are always at index 4, 5
gen_quarks = events.GenPart[events.GenPart.hasFlags(["isHardProcess"])][:, 4:6]

Matching efficiency calculation

In [ ]:
def matching_efficiency(gen_quarks, vbf_jets, matching_dr=0.4, verbose=False):
    drs = ak.pad_none(vbf_jets, 2, axis=1)[:, :2].metric_table(gen_quarks)
    matched = drs < matching_dr
    # TODO: add overlap removal?
    matching_fraction = np.mean(np.all(np.any(matched, axis=2), axis=1))
    if verbose:
        print(f"Matching efficiency: {matching_fraction}")
    return matching_fraction

In [ ]:
# ak8 jet preselection
preselection = {  # noqa: RUF012
    "pt": 300.0,
    "eta": 2.4,
    "VVmsd": 50,
    # "VVparticleNet_mass": [50, 250],
    # "bbparticleNet_mass": [92.5, 162.5],
    "bbparticleNet_mass": 50,
    "VVparticleNet_mass": 50,
    "bbFatJetParticleNetMD_Txbb": 0.8,
    "jetId": 2,  # tight ID bit
    "DijetMass": 800,  # TODO
    # "nGoodElectrons": 0,
}

In [ ]:
num_jets = 2
fatjets = events.FatJet

fatjets = ak.pad_none(
    fatjets[(fatjets.pt > 300) * (fatjets.isTight) * (np.abs(fatjets.eta) <= 2.4)], 2, axis=1
)

# particlenet xbb vs qcd
txbb = pad_val(
    fatjets.particleNetMD_Xbb / (fatjets.particleNetMD_QCD + fatjets.particleNetMD_Xbb),
    num_jets,
    axis=1,
)

# bb VV assignment
bb_mask = txbb[:, 0] >= txbb[:, 1]
bb_mask = np.stack((bb_mask, ~bb_mask)).T

In [ ]:
# Leptons
electrons = events.Electron
electrons = electrons[(electrons.pt > 5) & (electrons.cutBased >= electrons.LOOSE)]

muons = events.Muon
muons = muons[(muons.pt > 7) & (muons.looseId)]

In [ ]:
sel = ak.fill_none(
    (
        (txbb[bb_mask] > 0.97)
        * (fatjets.particleNet_H4qvsQCD[~bb_mask] > 0.6)
        * (fatjets.pt[:, 0] > 500)
        * (fatjets.pt[:, 1] > 400)
        * (np.abs(fatjets[:, 0].delta_phi(fatjets[:, 1])) > 2.6)
        * (np.abs(fatjets[:, 0].eta - fatjets[:, 1].eta) < 2.0)
    ),
    False,
)

In [ ]:
jets = events.Jet

# Final Selections

In [ ]:
ak4_jet_selection = {  # noqa: RUF012
    "pt_min": 15,  # was 25
    "pt_max": 640,  # was infty
    "eta_min": 0.25,  # was 0
    "eta_max": 4.9,  # was 4.7
    "jetId": "tight",
    "puId": "medium",
    "dR_fatjetbb": 1.0,  # was 1.2
    "dR_fatjetVV": 0.75,  # was 0.8
}

vbf_jet_mask = (
    jets.isTight
    & (jets.pt >= ak4_jet_selection["pt_min"])
    & (jets.pt <= ak4_jet_selection["pt_max"])
    & (np.abs(jets.eta) <= ak4_jet_selection["eta_max"])
    & (np.abs(jets.eta) >= ak4_jet_selection["eta_min"])
    & ((jets.pt > 50) | ((jets.puId & 2) == 2))
    & (
        ak.pad_none(fatjets, num_jets, axis=1, clip=True)[bb_mask].delta_r(jets)
        > ak4_jet_selection["dR_fatjetbb"]
    )
    & (
        ak.pad_none(fatjets, num_jets, axis=1, clip=True)[~bb_mask].delta_r(jets)
        > ak4_jet_selection["dR_fatjetVV"]
    )
    & ak.all(jets.metric_table(electrons) > 0.4, axis=2)
    & ak.all(jets.metric_table(muons) > 0.4, axis=2)
)


vbf_jets = jets[vbf_jet_mask]

eff = matching_efficiency(gen_quarks[sel], vbf_jets[sel], matching_dr=0.4)
print(f"Matching efficiency: {eff}")

## Comparison with an older cut

In [ ]:
ak4_jet_selection_old = {  # noqa: RUF012
    "pt_min": 25,
    "pt_max": np.inf,
    "eta_min": 0,
    "eta_max": 4.7,
    "jetId": "tight",
    "puId": "medium",
    "dR_fatjetbb": 1.2,
    "dR_fatjetVV": 0.8,
}

# dR_fatjetVV = 0.8 used from last two cells of VBFgenInfoTests.ipynb with data generated from SM signal vbf
# https://github.com/rkansal47/HHbbVV/blob/vbf_systematics/src/HHbbVV/VBF_binder/VBFgenInfoTests.ipynb
# (0-14R1R2study.parquet) has columns of different nGoodVBFJets corresponding to R1 and R2 cuts
vbf_jet_mask_old = (
    jets.isTight
    & (jets.pt >= ak4_jet_selection_old["pt_min"])
    & (jets.pt <= ak4_jet_selection_old["pt_max"])
    & (np.abs(jets.eta) <= ak4_jet_selection_old["eta_max"])
    & (np.abs(jets.eta) >= ak4_jet_selection_old["eta_min"])
    & ((jets.pt > 50) | ((jets.puId & 2) == 2))
    & (
        ak.pad_none(fatjets, num_jets, axis=1, clip=True)[bb_mask].delta_r(jets)
        > ak4_jet_selection["dR_fatjetbb"]
    )
    & (
        ak.pad_none(fatjets, num_jets, axis=1, clip=True)[~bb_mask].delta_r(jets)
        > ak4_jet_selection["dR_fatjetVV"]
    )
    & ak.all(jets.metric_table(electrons) > 0.4, axis=2)
    & ak.all(jets.metric_table(muons) > 0.4, axis=2)
)


vbf_jets_old = jets[vbf_jet_mask_old]

eff_old = matching_efficiency(gen_quarks[sel], vbf_jets_old[sel], matching_dr=0.4)
print(f"Matching efficiency: {eff_old}")

In [ ]:
def cut_efficiency(vbf_jets, threshold=3):
    etas = pad_val(vbf_jets.eta, 2, axis=1)
    mask = np.abs(etas[:, 0] - etas[:, 1]) > threshold
    return np.mean(mask)


print(f"Signal retain rate in new selection: {cut_efficiency(vbf_jets[sel])}")
print(f"Signal retain rate in old selection: {cut_efficiency(vbf_jets_old[sel])}")

In [ ]:
passing_per_event = ak.count(vbf_jets.pt, axis=1)
plt.figure(figsize=(8, 8))
plt.hist(passing_per_event, bins=np.arange(0, 9), histtype="step")
plt.title("Number of VBF-Tag Jets per Event")
plt.xlabel("Jets")
plt.ylabel("Events")
plt.savefig(plot_dir / "num_passing_vbf_jets.pdf", bbox_inches="tight")
plt.show()

# Effect on Background

In [ ]:
background = nanoevents.NanoEventsFactory.from_root(
    # a random QCD sample
    "root://cmseos.fnal.gov////store/user/lpcpfnano/cmantill/v2_3/2018/QCD/QCD_HT1500to2000_TuneCP5_PSWeights_13TeV-madgraph-pythia8/QCD_HT1500to2000_PSWeights_madgraph/220808_163124/0000/nano_mc2018_1-1.root",
    schemaclass=nanoevents.NanoAODSchema,
).events()

In [ ]:
higgs_bkg = background.GenPart[
    (abs(background.GenPart.pdgId) == HIGGS_PDGID) * background.GenPart.hasFlags(GEN_FLAGS)
]

higgs_bkg_children = higgs_bkg.children

# finding bb and VV children
is_bb_bkg = abs(higgs_bkg_children.pdgId) == b_PDGID
is_VV_bkg = (abs(higgs_bkg_children.pdgId) == W_PDGID) + (abs(higgs_bkg_children.pdgId) == Z_PDGID)

Hbb_bkg = higgs_bkg[ak.any(is_bb_bkg, axis=2)]
HVV_bkg = higgs_bkg[ak.any(is_VV_bkg, axis=2)]

# make sure we're only getting one Higgs
Hbb_bkg = ak.pad_none(Hbb_bkg, 1, axis=1)[:, 0]
HVV_bkg = ak.pad_none(HVV_bkg, 1, axis=1)[:, 0]

vs_bkg = background.GenPart[
    ((abs(background.GenPart.pdgId) == 24)) * background.GenPart.hasFlags(GEN_FLAGS)
]

# vbf output quarks are always at index 4, 5
gen_quarks_bkg = background.GenPart[background.GenPart.hasFlags(["isHardProcess"])][:, 4:6]

num_jets_bkg = 2
fatjets_bkg = background.FatJet

fatjets_bkg = ak.pad_none(
    fatjets_bkg[(fatjets_bkg.pt > 300) * (fatjets_bkg.isTight) * (np.abs(fatjets_bkg.eta) <= 2.4)],
    2,
    axis=1,
)

# particlenet xbb vs qcd
txbb_bkg = pad_val(
    fatjets_bkg.particleNetMD_Xbb / (fatjets_bkg.particleNetMD_QCD + fatjets_bkg.particleNetMD_Xbb),
    num_jets_bkg,
    axis=1,
)

# bb VV assignment
bb_mask_bkg = txbb_bkg[:, 0] >= txbb_bkg[:, 1]
bb_mask_bkg = np.stack((bb_mask_bkg, ~bb_mask_bkg)).T

# Leptons
electrons_bkg = background.Electron
electrons_bkg = electrons_bkg[
    (electrons_bkg.pt > 5) & (electrons_bkg.cutBased >= electrons_bkg.LOOSE)
]

muons_bkg = background.Muon
muons_bkg = muons_bkg[(muons_bkg.pt > 7) & (muons_bkg.looseId)]

sel_bkg = ak.fill_none(
    (
        (txbb_bkg[bb_mask_bkg] > 0.97)
        * (fatjets_bkg.particleNet_H4qvsQCD[~bb_mask_bkg] > 0.6)
        * (fatjets_bkg.pt[:, 0] > 500)
        * (fatjets_bkg.pt[:, 1] > 400)
        * (np.abs(fatjets_bkg[:, 0].delta_phi(fatjets_bkg[:, 1])) > 2.6)
        * (np.abs(fatjets_bkg[:, 0].eta - fatjets_bkg[:, 1].eta) < 2.0)
    ),
    False,
)

jets_bkg = background.Jet

bkg_jet_mask = (
    jets_bkg.isTight
    & (jets_bkg.pt >= ak4_jet_selection["pt_min"])
    & (jets_bkg.pt <= ak4_jet_selection["pt_max"])
    & (np.abs(jets_bkg.eta) <= ak4_jet_selection["eta_max"])
    & (np.abs(jets_bkg.eta) >= ak4_jet_selection["eta_min"])
    & ((jets_bkg.pt > 50) | ((jets_bkg.puId & 2) == 2))
    & (
        ak.pad_none(fatjets_bkg, num_jets_bkg, axis=1, clip=True)[bb_mask_bkg].delta_r(jets_bkg)
        > ak4_jet_selection["dR_fatjetbb"]
    )
    & (
        ak.pad_none(fatjets_bkg, num_jets_bkg, axis=1, clip=True)[~bb_mask_bkg].delta_r(jets_bkg)
        > ak4_jet_selection["dR_fatjetVV"]
    )
    & ak.all(jets_bkg.metric_table(electrons_bkg) > 0.4, axis=2)
    & ak.all(jets_bkg.metric_table(muons_bkg) > 0.4, axis=2)
)

bkg_jets = jets_bkg[bkg_jet_mask]

print(f"Background retain rate in new selection: {cut_efficiency(bkg_jets[sel_bkg])}")

# Selection analysis
We define true VBF jets as jets that are matched to the generator-level quarks. 
- We define fake VBF jets as jets that are not matched to the generator-level quarks.

We are interested in the following categories.
- `true_vbf_jets_selected`: true VBF jets that pass our selection (`vbf_jet_masked`).
- `true_vbf_jets_unselected`: true VBF jets that do not pass our selection (`vbf_jet_masked`).
- `fake_vbf_jets_selected`: jets that pass the selection (`vbf_jet_masked`) but are not true VBF jets.

In [ ]:
# jets that pass the VBF selection (without selecting the two highest pT jets yet)
jets = events.Jet
jets = jets[sel]

# ak4_jet_selection = {  # noqa: RUF012
#     "pt_min": 25,
#     "pt_max": np.inf,
#     "eta_min": 0,
#     "eta_max": 4.7,
#     "jetId": "tight",
#     "puId": "medium",
#     "dR_fatjetbb": 1.2,
#     "dR_fatjetVV": 0.8,
# }

ak4_jet_selection = {  # noqa: RUF012
    "pt_min": 15,  # was 25
    "pt_max": 640,  # was infty
    "eta_min": 0.25,  # was 0
    "eta_max": 4.9,  # was 4.7
    "jetId": "tight",
    "puId": "medium",
    "dR_fatjetbb": 1.0,  # was 1.2
    "dR_fatjetVV": 0.75,  # was 0.8
}

# ak4_jet_selection = {  # noqa: RUF012
#     "pt_min": 25,  # was 25
#     "pt_max": 640,  # was infty
#     "eta_min": 2.0,  # was 0
#     "eta_max": 4.9,  # was 4.7
#     "jetId": "tight",
#     "puId": "medium",
#     "dR_fatjetbb": 2.0,  # was 1.2
#     "dR_fatjetVV": 2.0,  # was 0.8
# }


vbf_jet_mask = (
    jets.isTight
    & (jets.pt >= ak4_jet_selection["pt_min"])
    & (jets.pt <= ak4_jet_selection["pt_max"])
    & (np.abs(jets.eta) <= ak4_jet_selection["eta_max"])
    & (np.abs(jets.eta) >= ak4_jet_selection["eta_min"])
    & ((jets.pt > 50) | ((jets.puId & 2) == 2))
    & (
        ak.pad_none(fatjets[sel], num_jets, axis=1, clip=True)[bb_mask[sel]].delta_r(jets)
        > ak4_jet_selection["dR_fatjetbb"]
    )
    & (
        ak.pad_none(fatjets[sel], num_jets, axis=1, clip=True)[~bb_mask[sel]].delta_r(jets)
        > ak4_jet_selection["dR_fatjetVV"]
    )
    & ak.all(jets.metric_table(electrons[sel]) > 0.4, axis=2)
    & ak.all(jets.metric_table(muons[sel]) > 0.4, axis=2)
)
jets_selected = jets[vbf_jet_mask]

# true jets
matching_dr = 0.4
drs = jets.metric_table(gen_quarks[sel])
matched = ak.any(drs < matching_dr, axis=2)

true_vbf_jets_selected = jets[vbf_jet_mask][:, :2][matched[vbf_jet_mask][:, :2]]
true_vbf_jets_unselected_by_ak4 = jets[~vbf_jet_mask][matched[~vbf_jet_mask]]
true_vbf_jets_unselected_by_2pt = jets[vbf_jet_mask][:, 2:][matched[vbf_jet_mask][:, 2:]]
false_vbf_jets_selected = jets[vbf_jet_mask][:, :2][~matched[vbf_jet_mask][:, :2]]

n_jets = len(ak.flatten(jets))
n_true_vbf_jets = len(ak.flatten(jets[matched]))
n_true_vbf_jets_selected = len(ak.flatten(true_vbf_jets_selected))
n_true_vbf_jets_selected_by_ak4 = len(ak.flatten(true_vbf_jets_unselected_by_ak4))
n_true_vbf_jets_selected_by_2pt = len(ak.flatten(true_vbf_jets_unselected_by_2pt))
n_false_vbf_jets_selected = len(ak.flatten(false_vbf_jets_selected))

print(f"Number of Jets: {n_jets}")
print(f"Number of True VBF Jets: {n_true_vbf_jets}")
print(
    f"Number of True VBF Jets Selected: {n_true_vbf_jets_selected} ({n_true_vbf_jets_selected/n_true_vbf_jets * 100:.2f}%)"
)
print(
    f"Number of True VBF Jets Unselected by ak4: {n_true_vbf_jets_selected_by_ak4} ({n_true_vbf_jets_selected_by_ak4/n_true_vbf_jets * 100:.2f}%)"
)
print(
    f"Number of True VBF Jets Unselected by pt: {n_true_vbf_jets_selected_by_2pt} ({n_true_vbf_jets_selected_by_2pt/n_true_vbf_jets * 100:.2f}%)"
)
print(f"Number of False VBF Jets Selected: {n_false_vbf_jets_selected}")
print(
    f"Fraction of True VBF Jets in Selected Jets: {n_true_vbf_jets_selected / (n_true_vbf_jets_selected + n_false_vbf_jets_selected) * 100:.2f}%"
)

In [ ]:
# mass
for ylabel, name_label, density in zip(
    ("Events", "Events (A.U)"),
    ("", "_density"),
    (False, True),
):
    plt.figure(figsize=(10, 10))
    bins = np.arange(0, 40, 1)
    plt.hist(
        ak.flatten(true_vbf_jets_selected.mass),
        bins=bins,
        histtype="step",
        density=density,
        label=f"True VBF Jets Selected ({n_true_vbf_jets_selected} Events)",
    )

    # plt.hist(
    #     ak.flatten(true_vbf_jets_unselected.mass),
    #     bins=bins,
    #     histtype="step",
    #     density=density,
    #     label=f"True VBF Jets Unselected ({n_true_vbf_jets_unselected} Events)",
    # )

    if n_true_vbf_jets_selected_by_ak4 > 0:
        plt.hist(
            ak.flatten(true_vbf_jets_unselected_by_ak4.mass),
            bins=bins,
            histtype="step",
            density=density,
            label=f"True VBF Jets Unselected by AK4 ({n_true_vbf_jets_selected_by_ak4} Events)",
        )

    if n_true_vbf_jets_selected_by_2pt > 0:
        plt.hist(
            ak.flatten(true_vbf_jets_unselected_by_2pt.mass),
            bins=bins,
            histtype="step",
            density=density,
            label=r"True VBF Jets Unselected by $p_\mathrm{T}$ "
            + f"({n_true_vbf_jets_selected_by_2pt} Events)",
        )

    plt.hist(
        ak.flatten(false_vbf_jets_selected.mass),
        bins=bins,
        histtype="step",
        density=density,
        label=f"False VBF Jets Selected ({n_false_vbf_jets_selected} Events)",
    )

    plt.legend(loc="upper right")
    plt.xlabel("$m_j$ (GeV)")
    plt.ylabel("ylabel")
    plt.savefig(plot_dir / f"vbf_jet_selection_mass{name_label}.pdf", bbox_inches="tight")
    plt.show()

In [ ]:
# pt
for ylabel, name_label, density in zip(
    ("Events", "Events (A.U)"),
    ("", "_density"),
    (False, True),
):
    plt.figure(figsize=(10, 10))
    bins = np.arange(0, 300, 10)
    plt.hist(
        ak.flatten(true_vbf_jets_selected.pt),
        bins=bins,
        histtype="step",
        density=density,
        label=f"True VBF Jets Selected ({n_true_vbf_jets_selected} Events)",
    )

    # plt.hist(
    #     ak.flatten(true_vbf_jets_unselected.pt),
    #     bins=bins,
    #     histtype="step",
    #     density=density,
    #     label=f"True VBF Jets Unselected ({n_true_vbf_jets_unselected} Events)",
    # )

    if n_true_vbf_jets_selected_by_ak4 > 0:
        plt.hist(
            ak.flatten(true_vbf_jets_unselected_by_ak4.pt),
            bins=bins,
            histtype="step",
            density=density,
            label=f"True VBF Jets Unselected by AK4 ({n_true_vbf_jets_selected_by_ak4} Events)",
        )

    if n_true_vbf_jets_selected_by_2pt > 0:
        plt.hist(
            ak.flatten(true_vbf_jets_unselected_by_2pt.pt),
            bins=bins,
            histtype="step",
            density=density,
            label=r"True VBF Jets Unselected by $p_\mathrm{T}$ "
            + f"({n_true_vbf_jets_selected_by_2pt} Events)",
        )

    plt.hist(
        ak.flatten(false_vbf_jets_selected.pt),
        bins=bins,
        histtype="step",
        density=density,
        label=f"False VBF Jets Selected ({n_false_vbf_jets_selected} Events)",
    )

    plt.legend(loc="upper right")
    plt.xlabel("${p_\mathrm{T}}_j$ (GeV)")
    plt.ylabel(ylabel)
    plt.savefig(plot_dir / f"vbf_jet_selection_pt{name_label}.pdf", bbox_inches="tight")
    plt.show()

In [ ]:
# |eta|
for ylabel, name_label, density in zip(
    ("Events", "Events (A.U)"),
    ("", "_density"),
    (False, True),
):
    plt.figure(figsize=(10, 10))
    bins = np.arange(0, 5, 0.1)
    plt.hist(
        np.abs(ak.flatten(true_vbf_jets_selected.eta)),
        bins=bins,
        histtype="step",
        density=density,
        label=f"True VBF Jets Selected ({n_true_vbf_jets_selected} Events)",
    )

    # plt.hist(
    #     np.abs(ak.flatten(true_vbf_jets_unselected.eta)),
    #     bins=bins,
    #     histtype="step",
    #     density=density,
    #     label=f"True VBF Jets Unselected ({n_true_vbf_jets_unselected} Events)",
    # )

    if n_true_vbf_jets_selected_by_ak4 > 0:
        plt.hist(
            np.abs(ak.flatten(true_vbf_jets_unselected_by_ak4.eta)),
            bins=bins,
            histtype="step",
            density=density,
            label=f"True VBF Jets Unselected by AK4 ({n_true_vbf_jets_selected_by_ak4} Events)",
        )

    if n_true_vbf_jets_selected_by_2pt > 0:
        plt.hist(
            np.abs(ak.flatten(true_vbf_jets_unselected_by_2pt.eta)),
            bins=bins,
            histtype="step",
            density=density,
            label=r"True VBF Jets Unselected by $p_\mathrm{T}$ "
            + f"({n_true_vbf_jets_selected_by_2pt} Events)",
        )

    plt.hist(
        np.abs(ak.flatten(false_vbf_jets_selected.eta)),
        bins=bins,
        histtype="step",
        density=density,
        label=f"False VBF Jets Selected ({n_false_vbf_jets_selected} Events)",
    )

    plt.legend(loc="upper right")
    plt.xlabel("$|\eta_j|$")
    plt.ylabel(ylabel)
    plt.savefig(plot_dir / f"vbf_jet_selection_eta{name_label}.pdf", bbox_inches="tight")
    plt.show()

In [ ]:
# eta_jj
true_vbf_jets_selected_padded = ak.pad_none(true_vbf_jets_selected, 2, axis=1)
# true_vbf_jets_unselected_padded = ak.pad_none(true_vbf_jets_unselected, 2, axis=1)
true_vbf_jets_unselected_by_ak4_padded = ak.pad_none(true_vbf_jets_unselected_by_ak4, 2, axis=1)
true_vbf_jets_unselected_by_2pt_padded = ak.pad_none(true_vbf_jets_unselected_by_2pt, 2, axis=1)
false_vbf_jets_selected_padded = ak.pad_none(false_vbf_jets_selected, 2, axis=1)

for ylabel, name_label, density in zip(
    ("Events", "Events (A.U)"),
    ("", "_density"),
    (False, True),
):
    plt.figure(figsize=(10, 10))
    bins = np.arange(1e-4, 10, 0.2)

    plt.hist(
        np.abs(true_vbf_jets_selected_padded.eta[:, 0] - true_vbf_jets_selected_padded.eta[:, 1]),
        bins=bins,
        histtype="step",
        density=density,
        label=f"True VBF Jets Selected ({n_true_vbf_jets_selected} Events)",
    )

    # plt.hist(
    #     np.abs(
    #         true_vbf_jets_unselected_padded.eta[:, 0] - true_vbf_jets_unselected_padded.eta[:, 1]
    #     ),
    #     bins=bins,
    #     histtype="step",
    #     density=density,
    #     label=f"True VBF Jets Unselected ({n_true_vbf_jets_unselected} Events)",
    # )

    if n_true_vbf_jets_selected_by_ak4 > 0:
        plt.hist(
            np.abs(
                true_vbf_jets_unselected_by_ak4_padded.eta[:, 0]
                - true_vbf_jets_unselected_by_ak4_padded.eta[:, 1]
            ),
            bins=bins,
            histtype="step",
            density=density,
            label=f"True VBF Jets Unselected by AK4 ({n_true_vbf_jets_selected_by_ak4} Events)",
        )

    if n_true_vbf_jets_selected_by_2pt > 0:
        plt.hist(
            np.abs(
                true_vbf_jets_unselected_by_2pt_padded.eta[:, 0]
                - true_vbf_jets_unselected_by_2pt_padded.eta[:, 1]
            ),
            bins=bins,
            histtype="step",
            density=density,
            label=r"True VBF Jets Unselected by $p_\mathrm{T}$ "
            + f"({n_true_vbf_jets_selected_by_2pt} Events)",
        )

    plt.hist(
        np.abs(false_vbf_jets_selected_padded.eta[:, 0] - false_vbf_jets_selected_padded.eta[:, 1]),
        bins=bins,
        histtype="step",
        density=density,
        label=f"False VBF Jets Selected ({n_false_vbf_jets_selected} Events)",
    )

    plt.legend(loc="upper right")
    plt.xlabel("$\eta_{jj}$")
    plt.ylabel(ylabel)
    plt.savefig(plot_dir / f"vbf_jet_selection_eta_jj{name_label}.pdf", bbox_inches="tight")
    plt.show()

In [ ]:
# dR(j, Hbb)
for ylabel, name_label, density in zip(
    ("Events", "Events (A.U)"),
    ("", "_density"),
    (False, True),
):
    plt.figure(figsize=(10, 10))
    bins = np.arange(0, 7, 0.2)
    plt.hist(
        ak.flatten(true_vbf_jets_selected.delta_r(Hbb[sel])),
        bins=bins,
        histtype="step",
        density=density,
        label=f"True VBF Jets Selected ({n_true_vbf_jets_selected} Events)",
    )

    # plt.hist(
    #     ak.flatten(true_vbf_jets_unselected.delta_r(Hbb[sel])),
    #     bins=bins,
    #     histtype="step",
    #     density=density,
    #     label=f"True VBF Jets Unselected ({n_true_vbf_jets_unselected} Events)",
    # )

    if n_true_vbf_jets_selected_by_ak4 > 0:
        plt.hist(
            ak.flatten(true_vbf_jets_unselected_by_ak4.delta_r(Hbb[sel])),
            bins=bins,
            histtype="step",
            density=density,
            label=f"True VBF Jets Unselected by AK4 ({n_true_vbf_jets_selected_by_ak4} Events)",
        )

    if n_true_vbf_jets_selected_by_2pt > 0:
        plt.hist(
            ak.flatten(true_vbf_jets_unselected_by_2pt.delta_r(Hbb[sel])),
            bins=bins,
            histtype="step",
            density=density,
            label=r"True VBF Jets Unselected by $p_\mathrm{T}$ "
            + f"({n_true_vbf_jets_selected_by_2pt} Events)",
        )

    plt.hist(
        ak.flatten(false_vbf_jets_selected.delta_r(Hbb[sel])),
        bins=bins,
        histtype="step",
        density=density,
        label=f"False VBF Jets Selected ({n_false_vbf_jets_selected} Events)",
    )

    plt.legend(loc="upper right")
    plt.xlabel("$\Delta R(j, Hbb)$")
    plt.ylabel(ylabel)
    plt.savefig(plot_dir / f"vbf_jet_selection_dR_Hbb{name_label}.pdf", bbox_inches="tight")
    plt.show()

In [ ]:
# dR(j, HVV)
for ylabel, name_label, density in zip(
    ("Events", "Events (A.U)"),
    ("", "_density"),
    (False, True),
):
    plt.figure(figsize=(10, 10))
    bins = np.arange(0, 7, 0.2)
    plt.hist(
        ak.flatten(true_vbf_jets_selected.delta_r(HVV[sel])),
        bins=bins,
        histtype="step",
        density=density,
        label=f"True VBF Jets Selected ({n_true_vbf_jets_selected} Events)",
    )

    # plt.hist(
    #     ak.flatten(true_vbf_jets_unselected.delta_r(HVV[sel])),
    #     bins=bins,
    #     histtype="step",
    #     density=density,
    #     label=f"True VBF Jets Unselected ({n_true_vbf_jets_unselected} Events)",
    # )

    if n_true_vbf_jets_selected_by_ak4 > 0:
        plt.hist(
            ak.flatten(true_vbf_jets_unselected_by_2pt.delta_r(HVV[sel])),
            bins=bins,
            histtype="step",
            density=density,
            label=f"True VBF Jets Unselected by AK4 ({n_true_vbf_jets_selected_by_ak4} Events)",
        )

    if n_true_vbf_jets_selected_by_2pt > 0:
        plt.hist(
            ak.flatten(true_vbf_jets_unselected_by_ak4.delta_r(HVV[sel])),
            bins=bins,
            histtype="step",
            density=density,
            label=r"True VBF Jets Unselected by $p_\mathrm{T}$ "
            + f"({n_true_vbf_jets_selected_by_2pt} Events)",
        )

    plt.hist(
        ak.flatten(false_vbf_jets_selected.delta_r(HVV[sel])),
        bins=bins,
        histtype="step",
        density=density,
        label=f"False VBF Jets Selected ({n_false_vbf_jets_selected} Events)",
    )

    plt.legend(loc="upper right")
    plt.xlabel("$\Delta R(j, HVV)$")
    plt.ylabel(ylabel)
    plt.savefig(plot_dir / f"vbf_jet_selection_dR_HVV{name_label}.pdf", bbox_inches="tight")
    plt.show()